# Introduction
***
Cet outil permet de concevoir un survolteur (boost converter) en prévisualisant l'efficacité à l'aide des caractéristiques des composantes choisies. La topologie de base étudiée est la suivante (hard switching):
![dcdc](assets/dcdc_boost.png)



# Entrées
***
Ici, on peut entrer les caractéristiques des composantes choisies pour le DCDC:

In [13]:
# Inductance
ind_value = 220 # en uH
res_serie = 12 # en mOhm


In [14]:
# Switching FET
sw_r_ds = 11 # en mOhm
sw_c_ds = 400 # en pF
sw_on_off_tr = 31 # en ns -> temps de transition on-off

In [15]:
# Rectifier FET
re_r_ds = 13 # en mOhm

# Équations
***

## Relation Vin/Vout
La relation entre Vin et Vout peut-être représentée par l'équation suivante:

$$\frac{Vin}{Vout} = 1 - D$$
<br>
$$où\ D = Duty\ Cycle\ (\%)$$






## Relation 2
